<a href="https://colab.research.google.com/github/leogoncalves/recinfo/blob/main/Trabalho_RecInfo_Anal%C3%ADse_de_sentimentos_dos_%C3%BAltimos_5_anos_das_Billboard_Hot_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# kaggle dataset: https://www.kaggle.com/dhruvildave/billboard-the-hot-100-songs
# spotify api https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features

In [ ]:
import pandas as pd
